In [97]:
import pandas as pd
from matplotlib import pyplot as plt

In [101]:
def add_hours(original_date, n=2):
    splits = original_date.split('T')
    time_parts = splits[1].split(':')
    new_hour = int(time_parts[0]) + n
    
    return splits[0] + 'T' + str(new_hour) + ':' + time_parts[1] + ':' + time_parts[2]

def add_minutes(original_date, n=2):
    splits = original_date.split('T')
    time_parts = splits[1].split(':')
    new_min = int(time_parts[1]) + n
    
    return splits[0] + 'T' + time_parts[0] + ':' + str(new_min)  + ':' + time_parts[2]

def subtract_minutes(original_date, n=2):
    splits = original_date.split('T')
    time_parts = splits[1].split(':')
    new_min = int(time_parts[1]) - n
    
    return splits[0] + 'T' + time_parts[0] + ':' + str(new_min)  + ':' + time_parts[2]

def add_seconds(original_date, n=2):
    splits = original_date.split('T')
    time_parts = splits[1].split(':')
    new_sec = int(time_parts[2]) + n
    
    return splits[0] + 'T' + time_parts[0] + ':' + time_parts[1] + ':' + str(new_sec)

In [102]:
gpu_info = pd.read_csv('experiments.csv', index_col='id')

for index, row in gpu_info.iterrows():
    gpu_info.at[index,'start-time'] = add_hours(row['start-time'], 2)
    gpu_info.at[index,'end-time'] = add_hours(row['end-time'], 2)

In [103]:
grafana_gpu_power_usage = pd.read_csv('grafana_gpu_power_usage.csv', sep=';')

In [104]:
grafana_gpu_utilization = pd.read_csv('grafana_gpu_utilization.csv', sep=';')

In [105]:
grafana_gpu_memory = pd.read_csv('grafana_gpu_memory.csv', sep=';')

In [114]:
def get_grafana_info(nvidia_plugin, start_time, end_time, grafana_df, tolerance_min=15):
    start_time = subtract_minutes(start_time, tolerance_min)
    end_time = add_minutes(end_time, tolerance_min)
    res_df = grafana_df.loc[(grafana_df['Series'] == nvidia_plugin) & \
                            (grafana_df['Time'] > start_time) & \
                            (grafana_df['Time'] <= end_time)]
    values = res_df['Value'].values
    #plt.scatter(range(0, len(values)), values)
    #plt.show()
    return values
    
print(get_grafana_info('nvidia-gpu-device-plugin-j8nl9' + '-gpu0', 
                       '2020-09-09T13:51:13',
                       '2020-09-09T13:52:03',
                       grafana_gpu_utilization))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 60.  0.  0.
  0.  0.  0.  0.  0.]


In [126]:
for index, row in gpu_info.iterrows():
    print(row['network'], row['batch-size'])
    gpu_util = get_grafana_info(row['nvidia-plugins'] + '-gpu0', \
                                row['start-time'], \
                                row['end-time'], \
                                grafana_gpu_utilization)
    if all(v == 0 for v in gpu_util):
        print('ERROR')
    else:
        print('GPU_util = ' + str(max(gpu_util)))
        
    gpu_util = get_grafana_info(row['nvidia-plugins'] + '-gpu0', \
                                row['start-time'], \
                                row['end-time'], \
                                grafana_gpu_memory)
    if all(v == 0 for v in gpu_util):
        print('ERROR')
    else:
        print('MEM = ' + str(max(gpu_util)))
        
    gpu_util = get_grafana_info(row['nvidia-plugins'] + '-gpu0', \
                                row['start-time'], \
                                row['end-time'], \
                                grafana_gpu_power_usage)
    if all(v == 0 for v in gpu_util):
        print('ERROR')
    else:
        print('Power = ' + str(max(gpu_util)))
        
    print()

resnet 32
GPU_util = 60.0
MEM = 95.37514802909828
Power = 53985.0

alexnet 32
GPU_util = 6.0
MEM = 95.37514802909828
Power = 41608.0

inception 32
GPU_util = 98.0
MEM = 95.37514802909828
Power = 255608.0

resnet 64
ERROR
MEM = 95.35054059457714
Power = 169985.0

alexnet 64
ERROR
MEM = 95.37514802909828
Power = 35903.0

inception 64
GPU_util = 99.0
MEM = 95.37514802909828
Power = 224642.0

resnet 128
GPU_util = 99.0
MEM = 95.38129988772856
Power = 220230.0

alexnet 128
GPU_util = 9.0
MEM = 95.38129988772856
Power = 41806.0

inception 128
GPU_util = 100.0
MEM = 95.37514802909828
Power = 252247.0

resnet 256
GPU_util = 100.0
MEM = 95.37514802909828
Power = 228526.0

alexnet 256
GPU_util = 3.0
MEM = 95.38129988772856
Power = 140986.0

alexnet 512
GPU_util = 56.0
MEM = 95.38129988772856
Power = 107544.0

alexnet 1024
GPU_util = 30.0
MEM = 95.38745174635885
Power = 253805.0

alexnet 2048
GPU_util = 100.0
MEM = 95.39975546361946
Power = 251548.0

